## Commuting distance per occupation: analysis (using residents 2-digit SOC)
This notebook is to analyse the average distance travelled per occupation across different travel to work areas (TTWA). For each output area (OA), or lower super output area (LSOA) in a TTWA, collected using the LMI for ALL API.

The centroids for each OA, or LSOA, are computed in another script (get_oa_lsoa_centroids) using the ONS postcode directory (February 2019). The same dataset also offers a lookup between OAs and TTWAs.

In [ ]:
import urllib.request, json
import pandas as pd
import numpy as np
import pickle
import time
import matplotlib.pyplot as plt
import os
import seaborn as sns
import copy
from importlib import reload
from pandas.plotting import table
# set the default to darkgrid
sns.set_style('darkgrid')

In [ ]:
import os
import itertools
import pandas as pd
import numpy as np 
import pickle
import collections
import seaborn as sns
from sklearn import preprocessing

# Python tools
from matplotlib import pyplot as plt
import math
from collections import defaultdict
from functools import partial

# plotly
import plotly
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)


In [ ]:
%matplotlib inline

In [ ]:
# import all filenames (stored in a file that is in common to multiple scripts)
import all_filenames
from all_filenames import *

In [ ]:
import utils_pin
from utils_pin import print_elapsed, draw_map, draw_map_and_landmarks

In [ ]:
# plot saving folder
plot_save_dir = '/Users/stefgarasto/Google Drive/Documents/results/PIN/plots/'
# file where I'm storing all the information
save_oa_file = folder3 + 'PIN/oa_distances_and_occupations_v2.pickle'
save_oa_file_jobs = folder3 + 'PIN/oa_jobs_breakdown.pickle'
FIGSAVE = False

In [ ]:
# first, load the list of all TTWA
ttwa_data = pd.read_csv(ttwa_file)
# first column is ttwa codes, second column is ttwa names
ttwa_info11 = pd.read_excel(ttwa_info11_file)
ttwa_info16 = pd.read_excel(ttwa_info16_file)
#print(ttwa_info11.tail(n=3))
#print(ttwa_info16.tail(n=3))

# get small TTWAs
small_ttwas = list(ttwa_info11['ttwa11cd'][ttwa_info11['LSOAs']<40])
print('There are {} TTWAs with less than 40 LSOAs.'.format(len(small_ttwas)))

# also, eliminate three scottish TTWAs that seem to mostly have nans:
# just joking: can't eliminate Edinburgh and Aberdeen
#bad_ttwas = []#'S22000059', 'S22000047', 'S22000071']
small_ttwas = small_ttwas #+ bad_ttwas

# now set the ttwa code as the index
ttwa_data = ttwa_data.set_index('ttwa11cd')
ttwa_info11 = ttwa_info11.set_index('ttwa11cd')
ttwa_info16 = ttwa_info16.set_index('ttwa11cd')

# drop rows
ttwa_data = ttwa_data.drop(small_ttwas, axis = 0)
ttwa_info11 = ttwa_info11.drop(small_ttwas, axis = 0)
ttwa_info16 = ttwa_info16.drop([t for t in small_ttwas if t in ttwa_info16.index], axis = 0)
print(len(ttwa_data),len(ttwa_info11),len(ttwa_info16))
ttwa_data = ttwa_data.sort_index()
ttwa_info16 = ttwa_info16.sort_index()
ttwa_info11 = ttwa_info11.sort_index()

# Create aliases for the column names (need to be shorter to be plotted correctly)
rename_cols16 = {'Employment rate ': 'Employment rate',
       '% of economically inactive who want a job':'Job-seeking economically inactive',
       'Claimant Count, % aged 16-64, April 2015 to March 2016 ': 'Claimant count',
       'All in employment who are 1: managers, directors and senior officials (SOC2010)': 'Employed in SOC code 1',
       ' All in employment who are 2: professional occupations or 3: associate prof & tech occupations (SOC2010)': 
                 'Employed in SOC code 2',
       'All in employment who are 5: skilled trades occupations (SOC2010)': 'Employed in SOC code 5',
       'All in employment who are 6: caring, leisure and other service occupations (SOC2010)': 
                 'Employed in SOC code 6',
       'All in employment who are 8: process, plant and machine operatives (SOC2010)':'Employed in SOC code 8',
       'All in employment who are 9: elementary occupations (SOC2010)':'Employed in SOC code 9'}

rename_cols11 = {'Supply-side self-containment (% employed residents who work locally)':
                 'Supply-side self-containment',
       'Demand-side self-containment (% local jobs taken by local residents)':'Demand-side self containment',
       'Number of economically active residents (aged 16+)':'Economically active residents'}
#ttwa_info16 = 
ttwa_info16.rename(rename_cols16, axis = 1, inplace = True)
#ttwa_info11 = 
ttwa_info11.rename(rename_cols11, axis = 1, inplace = True)



In [ ]:
# load the extracted dictionaries of OA centroids
loadOA = True
loadLSOA = False
oa_path = folder4 + 'oa_centroids_dictionary.pickle'
lsoa_path = folder4 + 'lsoa_centroids_dictionary.pickle'
exists = os.path.isfile(oa_path)
if exists and loadOA:
    print('Loading the OA data')
    oa_data = pd.read_pickle(oa_path)

exists = os.path.isfile(lsoa_path)
if exists and loadLSOA:
    print('Loading the LSOA data')
    lsoa_data = pd.read_pickle(lsoa_path)


In [ ]:
# Load the data dictionaries which then should be transformed to dataframes and joined.
# They should also be joined with the list of TTWAs for each OA
# Then, I can make the relevant plots
# What I want is a breakdown of mean travel distances for occupations and for ttwa

# first, load the data
with open(save_oa_file, 'rb') as f:
    _,oa_distances,oa_occupations,oa_residents,socGroups,_,_ = pickle.load(f)


# create the dataframes from the dictionaries and join the data frames
t0 = time.time()
#oa_frame = pd.DataFrame.from_dict(oa_distances, orient = 'index').join(
#    pd.DataFrame.from_dict(oa_occupations, orient = 'index')).join(
#    pd.DataFrame.from_dict(oa_residents, orient = 'index')).join(oa_data)
oa_frame = pd.DataFrame.from_dict(oa_distances, orient = 'index').join(
    pd.DataFrame.from_dict(oa_occupations, orient = 'index')).join(
    pd.DataFrame.from_dict(oa_residents, orient = 'index')).join(oa_data)
print('It took {:2f}s to create the full dataframe with {} rows'.format(time.time()- t0, len(oa_frame)))
# rename the residents column
oa_frame.rename(columns = {0: 'residents'}, inplace = True)
# What I should have just created is a dataframe that should have each OA as its index, while the columns are:
# breakdown of mean distances (all, male, female, age groups)
# breakdown of occupations per two digit SOC groups (i.e. how many workers in each TTWA for each occupation, in 
# absolute and relative values)
# latitude, longitude and ttwa
print(oa_frame.columns,socGroups)

In [ ]:
# group by TTWAs
groups = oa_frame.groupby('ttwa')
# get all the TTWAs - however, exclude the ones that are very small since the data might be inaccurate
all_ttwas = list(ttwa_data.index) #list(set(groups.groups.keys()) - set(small_ttwas))
print(len(all_ttwas))
# Here I'm hardcoding all the occupations
all_occupations = ['11','12','21','22','23','24','31','32','33','34','35','41','42',
                   '51','52','53','54','61','62','71','72','81','82']

In [ ]:
# compute the mean distance per occupation per TTWA, by computing a sum over OAs in a given TTWA weighted by the
# percentange of residents in that OA that work in a specific occupation
M_dist = np.zeros((len(all_occupations),len(all_ttwas)))
for jj,ttwa in enumerate(all_ttwas):
    # get the group corresponding to this TTWA
    group = groups.get_group(ttwa)
    # collect the mean distances to work
    dks = np.array(group['Mean distance to work (overall)'])
    # go through all the occupations
    for ii,occ in enumerate(all_occupations):
        # get the weights, given by the normalised number of how many residents are in this occupation
        weights= np.array(group[occ + '_percentage'])
        # set to 0 any weight that is nan
        weights[np.isnan(weights)]=0
        weights = weights/np.sum(weights)
        # multiply the distances by the normalised weights and sum over the OAs
        M_dist[ii,jj] = np.sum(dks * weights)

# TODO: I'm pretty sure this can be optimised, and even left in a dataframe so that I can use seaborn
# what I would want is one column with TTWAs, one with occupation codes, one with the data

# for now, replace nans with mean over column
col_mean = np.nanmean(M_dist, axis=0)
'''
# Find indices that I need to replace
inds = np.where(np.isnan(M_dist))
# Place column means in the indices. Align the arrays using take
M_dist[inds] = np.take(col_mean, inds[1])
'''
# for now, also drop TTWA whose mean is nan (there should be none)
inds_ttwa = np.where(~np.isnan(col_mean))[0]
M_dist = M_dist[:,inds_ttwa]

# collect all names and codes:
ttwa_names = [ttwa_data['ttwa11nm'][ttwa_data.index==t].values[0] for t in all_ttwas]


In [ ]:
# plot for first half of the TTWAs
inds_plot = range(0,len(inds_ttwa)//2)
fig = plt.figure(figsize= (12,20))
fig.add_subplot(1,2,1)
ttwa_labels= [all_ttwas[ii] for ii in inds_ttwa[inds_plot]]
ttwa_names_plot = [ttwa_data['ttwa11nm'][ttwa_data.index==t].values[0] for t in ttwa_labels]
ax = plt.boxplot(M_dist[:,inds_plot], vert = 0, labels = ttwa_labels)
#locs, labels = plt.yticks() 
#plt.yticks(locs, ttwa_labels, fontsize = 12) #, rotation = 'vertical')
plt.ylabel('TTWA', fontsize = 12)
tmp = plt.xlabel('Average commuting distance', fontsize = 12)
# add text 
pos = range(len(inds_ttwa)//2)
for label,tick in zip(pos,ax['medians']):
    plt.text(tick.get_xdata()[0], tick.get_ydata()[1] + 0.03, ttwa_names_plot[label],
            horizontalalignment='center',  color='k', weight='semibold') #size='x-small',
plt.tight_layout()
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_by_TTWA_pt1.png'))
    
#second half
inds_plot = range(len(inds_ttwa)//2,len(inds_ttwa))
fig.add_subplot(1,2,2)
ttwa_labels= [all_ttwas[ii] for ii in inds_ttwa[inds_plot]]
ttwa_names_plot = [ttwa_data['ttwa11nm'][ttwa_data.index==t].values[0] for t in ttwa_labels]
ax = plt.boxplot(M_dist[:,inds_plot], vert = 0, labels = ttwa_labels)
#locs, labels = plt.yticks() 
#plt.yticks(locs, [all_ttwas[ii] for ii in inds_ttwa[inds_plot]], fontsize = 12) #, rotation = 'vertical')
plt.ylabel('TTWA', fontsize = 12)
tmp = plt.xlabel('Average commuting distance', fontsize = 12)
# add text 
pos = range(len(inds_ttwa)//2)
for label,tick in zip(pos,ax['medians']):
    plt.text(tick.get_xdata()[0], tick.get_ydata()[1] + 0.03, ttwa_names_plot[label],
            horizontalalignment='center',  color='k', weight='semibold') #size='x-small',
plt.tight_layout()
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_by_TTWA_pt2.png'))

In [ ]:
'''
# plot a quarter/third of the TTWAs
inds_plot = range(90,135)
fig = plt.figure(figsize= (16,4))
plt.boxplot(M_dist[:,inds_plot])
locs, labels = plt.xticks() 
plt.xticks(locs, [all_ttwas[ii] for ii in inds_ttwa[inds_plot]], fontsize = 12, rotation = 'vertical')
plt.xlabel('TTWA', fontsize = 12)
tmp = plt.ylabel('Average commuting distance', fontsize = 12)
'''
print('Not needed if excluding small TTWAs')

In [ ]:
# Add mean distance across TTWAs to ttwa info dataframes
t0 = time.time()
ttwa_info11['mean_distance'] = np.nan
for row in ttwa_info11.index:
    if row in all_ttwas:
        ix = all_ttwas.index(row)
        ttwa_info11['mean_distance'].loc[row] = np.mean(M_dist[:,ix])
        if np.isnan(np.mean(M_dist[:,ix])):
            print(row, ' is NAN.')
    else:
        continue
#ttwa_info11['mean_distance'] = np.mean(M_dist, axis = 0)
ttwa_info16['mean_distance'] = ttwa_info11['mean_distance']
print_elapsed(t0,'joining mean distances and summary statistics')

In [ ]:
print(plt.style.available)


In [ ]:
def remove_london(x):
    try:
        if x > 2000000:
            return np.nan
        else:
            return x
    except:
        return x

# plot all correlations between means and the other variables
plt.style.use('seaborn-deep')
ttwa_info16 = ttwa_info16.replace('!',np.nan)
fig, ax = plt.subplots(6,3,figsize=(13,16))
for t,col in enumerate(ttwa_info16.columns[2:20]):
    with sns.plotting_context('talk'):
        i,j = np.unravel_index(t,(6,3))
        #ttwa_info16.plot.scatter(y = 'mean_distance',x = col, ax = ax[i,j])
        sns.scatterplot(data=ttwa_info16.applymap(remove_london),y = 'mean_distance',x = col, ax = ax[i,j])
        rho = ttwa_info16['mean_distance'].corr(ttwa_info16[col])
        ax[i,j].set_xlabel(col + '. rho = {:.4f}'.format(rho), fontsize = 12)
        ax[i,j].set_ylabel(ax[i,j].get_ylabel(), fontsize = 12)
        ax[i,j].tick_params(axis = 'both', labelsize = 12)
plt.tight_layout()

if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_vs_ttwa_stats16.png'))


In [ ]:
def remove_london(x):
    try:
        if x > 2000000:
            return np.nan
        else:
            return x
    except:
        return x

# redo: plot all correlations between means and the other variables
main_col16 = ['Male employment rate', 'Female employment rate','Claimant count',
              ' Population','Employed in SOC code 1','Employed in SOC code 9']

plt.style.use('seaborn-deep')
ttwa_info16 = ttwa_info16.replace('!',np.nan)
fig, ax = plt.subplots(2,3,figsize=(13,6))
for t,col in enumerate(main_col16):
    with sns.plotting_context('talk'):
        i,j = np.unravel_index(t,(2,3))
        #ttwa_info16.plot.scatter(y = 'mean_distance',x = col, ax = ax[i,j])
        sns.scatterplot(data=ttwa_info16.applymap(remove_london),y = 'mean_distance',x = col, ax = ax[i,j])
        rho = ttwa_info16['mean_distance'].corr(ttwa_info16[col])
        ax[i,j].set_xlabel(col + '. rho = {:.4f}'.format(rho), fontsize = 12)
        ax[i,j].set_ylabel(ax[i,j].get_ylabel(), fontsize = 12)
        ax[i,j].tick_params(axis = 'both', labelsize = 12)
plt.tight_layout()

if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_vs_ttwa_stats16_select.png'))


In [ ]:
# plot all correlations between means and the other variables

ttwa_info11 = ttwa_info11.replace('!',np.nan)
fig, ax = plt.subplots(4,2,figsize=(12,16))
for t,col in enumerate(ttwa_info11.columns[2:9]):
    with sns.plotting_context('talk'):
        i,j = np.unravel_index(t,(4,2))
        ttwa_info11['tmp'] = ttwa_info11[col]
        ttwa_info11['tmp'] = ttwa_info11['tmp'].map(remove_london)
        sns.scatterplot(data=ttwa_info11,y = 'mean_distance',x = 'tmp', ax = ax[i,j])
        rho = ttwa_info11['mean_distance'].corr(ttwa_info11['tmp'])
        ax[i,j].set_xlabel(col + '. rho = {:.4f}'.format(rho), fontsize = 12)
        ax[i,j].set_ylabel(ax[i,j].get_ylabel(), fontsize = 12)
        ax[i,j].tick_params(axis = 'both', labelsize = 12)
plt.tight_layout
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_vs_ttwa_stats11.png'))

In [ ]:
# redo: plot all correlations between means and the other variables for selected columns

main_col11 = ['Supply-side self-containment', 'Demand-side self containment',
       'Land Area (hectares)']
ttwa_info11 = ttwa_info11.replace('!',np.nan)
fig, ax = plt.subplots(1,3,figsize=(13,3))
for t,col in enumerate(main_col11):
    with sns.plotting_context('talk'):
        i,j = np.unravel_index(t,(1,3))
        ttwa_info11['tmp'] = ttwa_info11[col]
        ttwa_info11['tmp'] = ttwa_info11['tmp'].map(remove_london)
        sns.scatterplot(data=ttwa_info11,y = 'mean_distance',x = 'tmp', ax = ax[j])
        rho = ttwa_info11['mean_distance'].corr(ttwa_info11['tmp'])
        ax[j].set_xlabel(col + '. rho = {:.4f}'.format(rho), fontsize = 12)
        ax[j].set_ylabel(ax[j].get_ylabel(), fontsize = 12)
        ax[j].tick_params(axis = 'both', labelsize = 12)
        if col == 'Economically active residents':
            ax[j].tick_params(axis = 'x', labelrotation = 45)
plt.tight_layout
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_vs_ttwa_stats11_select.png'))

In [ ]:
fig = plt.figure(figsize = (8,5))
g = sns.scatterplot(x='Supply-side self-containment',y='Demand-side self containment',
               hue = 'mean_distance', data= ttwa_info11, sizes = [20], 
                palette = 'YlGnBu_r', s = 80)
ax = plt.gca()
ax.set_ylabel(ax.get_ylabel(),fontsize = 14)
ax.set_xlabel(ax.get_xlabel(),fontsize = 14)
ax.tick_params(axis = 'both', labelsize = 13)

In [ ]:
# plot same plot as above but interactive
reload(plotly)
PuOr = sns.color_palette(palette= 'PuOr', n_colors = 256)

fig = {
    'data': [
          {
          'x': ttwa_info11['Supply-side self-containment'], 
            'y': ttwa_info11['Demand-side self containment'], 
            'text': ttwa_info11['ttwa11nm'], 
            'mode': 'markers', 
            'name': 'Demand vs Supply',
            'marker': dict(
                size=16,
                color = ttwa_info11['mean_distance'], #set color equal to a variable
                colorscale='YlGnBu',
                showscale=True
                )
          },
    ],
    'layout': {
        'xaxis': {'title': 'Supply-side self-containment'},
        'yaxis': {'title': "Demand-side self-containment"}
    }
}
plotly.offline.iplot(fig, filename = os.path.join(plot_save_dir,'demand_vs_supply_interactive.html'))
if FIGSAVE:
    plotly.offline.plot(fig, filename = os.path.join(plot_save_dir,'demand_vs_supply_interactive.html'), 
                    auto_open = False)
                #, image = 'png', image_filename = os.path.join(plot_save_dir,'demand_vs_supply_interactive.html'))
#plotly.io.write_image(fig, file = os.path.join(plot_save_dir,'demand_vs_supply_interactive.png'))
#py.iplot(fig, filename='pandas-multiple-scatter')

In [ ]:
show_cols = ['ttwa11nm','Region/Country', ' Population', 'Employment rate','Economically inactive',
             'Job-seeking economically inactive',
             'Employed in SOC code 1',
            'Employed in SOC code 1','mean_distance']
#for col in show_cols[1:]:
#    print(ttwa_info16[col].mean())
    # need to change a few strings to nans
print('The averages for each column are:')
print([(col, ttwa_info16[col].mean()) for col in show_cols[2:]])
print('\n')
print('The 10 TTWA with the shortest distance are: ')
ttwa_info16[show_cols].sort_values('mean_distance')[:10]



In [ ]:
print('The 10 TTWA with the highest distance are: ')
ttwa_info16[show_cols].sort_values('mean_distance')[-10:]


In [ ]:
def assign_score(x):
    if x['best']:
        return 'Shortest'
    elif x['worst']: #TTWA'] in highest_distance_ttwas:
        return 'Longest'
    else:
        return 'Average'
    
def assign_region(row,ttwa_info):
    region = ttwa_info['Region/Country'].loc[row['TTWA']]
    return region

In [ ]:
tmp = ttwa_info16.sort_values('mean_distance')['mean_distance'].values
p = np.percentile(ttwa_info16['mean_distance'].values, 20)
np.sum(tmp<p)/tmp.size
N = int(np.round(len(ttwa_info16)/4))
print(N)

In [ ]:
# now get the lowest and highest 25th percentile - that is take the bottom and top quarter
N = int(np.round(len(ttwa_info16)/4))
shortest_distance_ttwas = ttwa_info16.sort_values('mean_distance')[:N]['ttwa11nm'].values
shortest_distance_codes = ttwa_info16.sort_values('mean_distance').index[:N].values
highest_distance_ttwas = ttwa_info16[show_cols].sort_values('mean_distance')[-N:]['ttwa11nm'].values
highest_distance_codes = ttwa_info16[show_cols].sort_values('mean_distance').index[-N:].values

In [ ]:
# add best and worst indicator to ttwa_info11 and ttwa_info16 too
ttwa_info11 = ttwa_info11.assign(best = [t in shortest_distance_codes for t in ttwa_info11.index])
ttwa_info11 = ttwa_info11.assign(worst = [t in highest_distance_codes for t in ttwa_info11.index])
ttwa_info11['Rank'] = ttwa_info11.apply(assign_score, axis = 1)
# ttwa_info16
#ttwa_info16['best'] = ttwa_info16.reset_index()['ttwa11cd'].apply(lambda x: x in shortest_distance_codes)
ttwa_info16 = ttwa_info16.assign(best = [t in shortest_distance_codes for t in ttwa_info16.index])
ttwa_info16 = ttwa_info16.assign(worst = [t in highest_distance_codes for t in ttwa_info16.index])
ttwa_info16['Rank'] = ttwa_info16.apply(assign_score, axis = 1)
print('Done')

In [ ]:
# transform the data into a dataframe. The column that stores the TTWA codes is called TTWA
t0 = time.time()
select_ttwa = [all_ttwas[ii] for ii in inds_ttwa]
select_ttwa_names = [ttwa_data['ttwa11nm'][ttwa_data.index==t].values[0] for t in select_ttwa]
distances_data = pd.DataFrame(M_dist.T, index = select_ttwa, 
                    columns = all_occupations)
distances_data['TTWA'] = distances_data.index
distances_data['TTWA name'] = ttwa_info11['ttwa11nm']

# make a copy of the dataframe wide form
distances_data_wide = copy.deepcopy(distances_data)
distances_data_wide['Region'] = distances_data_wide['TTWA'].apply(lambda x: ttwa_info16['Region/Country'].loc[x])

# transform the dataframe into a long form
distances_data = distances_data.melt(id_vars = ['TTWA','TTWA name'], var_name = ['Occupation code'],
                                    value_name = 'Mean distance')
distances_data['Occupation first digit'] = distances_data['Occupation code'].apply(lambda x: x[0])
distances_data['best'] = distances_data['TTWA'].apply(lambda x: x in shortest_distance_codes)
distances_data['worst'] = distances_data['TTWA'].apply(lambda x: x in highest_distance_codes)
distances_data['Rank'] = distances_data.apply(assign_score, axis = 1)
distances_data['Region'] = distances_data.apply(lambda x: assign_region(x, ttwa_info16), axis = 1)
#distances_data['ttwa'].value_counts()
#len(all_occupations)
print(time.time() - t0, len(distances_data))
distances_data.head(n=3)


In [ ]:
# plot breakdown of average commuting distances by SOC code (first digit) and regions
plt.style.use('seaborn-deep')

with sns.plotting_context('talk'):
    g = sns.catplot(data = distances_data, y = 'Mean distance', x = 'Region', hue = 'Occupation first digit', 
                    kind = 'box', height = 5, aspect = 18/5, order = ['Yorkshire and The Humber', 'South West', 
                    'East Midlands', 'South East','North West','North East','East of England',
                    'West Midlands','Scotland','Wales','NI'], legend_out = True)
    plt.gca().tick_params(axis = 'x', labelrotation = 45)
    ax = plt.gca()
    for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)

#plt.tight_layout()
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_by_region_and_1d-SOC.png'),bbox_inches = 'tight')
    
with sns.plotting_context('talk'):
    g = sns.catplot(data = distances_data, y = 'Mean distance', x = 'Region', hue = 'Occupation first digit', 
                    kind = 'box', height = 5, aspect = 1, order = ['London'], legend_out = True)
    ax = plt.gca()
    for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(14)

if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_london_and_1d-SOC.png'),bbox_inches = 'tight')

print(distances_data['Region'].value_counts()/23)

In [ ]:
def mypercentile(n):
    def percentile_(x):
        return x.quantile(n)
    percentile_.__name__ = 'percentile_{:2.0f}'.format(n*100)
    return percentile_

# collect some stats for each region
tmp = distances_data.groupby('Region')['Mean distance'].aggregate([
    mypercentile(.25), np.median, mypercentile(.75),np.std])
tmp = tmp.applymap(lambda x: np.around(x,3))

# prepare to plot the table: make the rest of the plot invisible
ax = plt.subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

# make the table and make it bigger
Tab = table(ax, tmp, fontsize= 16)
Tab.scale(2,2)
Tab.set_fontsize(16)
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_by_region_stats.png'),bbox_inches = 'tight')

In [ ]:
with sns.plotting_context('talk'):
    g = sns.catplot(data = distances_data, y = 'Mean distance', x = 'Rank', hue = 'Occupation first digit', 
                    kind = 'box', height = 5, aspect = 18/5)
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_by_ranking_and_1d-SOC.png'))

In [ ]:
# try plotting various indicators still comparing by best - worst - average TTWA
fig, ax = plt.subplots(4,4,figsize=(12,12))
with sns.plotting_context('talk', font_scale = 3):
    for t,col in enumerate(list(ttwa_info16.columns[3:12])+list(ttwa_info16.columns[13:20])):
        i,j = np.unravel_index(t,(4,4))
        g = sns.boxplot(data = ttwa_info16, y = col, x = 'Rank', ax = ax[i,j]) 
                    #kind = 'box', height = 4, aspect = 1.2)
        #ttwa_info16.plot.scatter(y = 'mean_distance',x = col, ax = ax[i,j])
        #ax[i,j].set_ylabel(col[0:28])
    # now plot the regions
    i,j = np.unravel_index(t+1,(5,4))
plt.tight_layout()
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_by_ranking_and_stats16_pt1.png'))
    
fig = plt.figure(figsize=(8,4))
g = sns.countplot(data = ttwa_info16, x = 'Region/Country', hue = 'Rank')
g.tick_params(axis = 'x', labelrotation = 90)
g.tick_params(axis = 'both', labelsize = 13)
g.set_xlabel(g.get_xlabel(),fontsize = 14)
g.set_ylabel(g.get_ylabel(),fontsize = 14)
plt.tight_layout
'''
with sns.plotting_context('talk'):
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
    sns.boxplot(x='not.fully.paid', y='int.rate', kind='box', data=loans, ax=ax1)
    sns.boxplot(x='not.fully.paid', y='fico', kind='box', data=loans, ax=ax2)
    g = sns.catplot(data = ttwa_info16, y = 'Employment rate ', x = 'order', 
                    kind = 'box', height = 4, aspect = 1.2)
'''
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_by_ranking_and_stats16_pt2.png'))
    

In [ ]:
# try plotting various indicators still comparing by best - worst - average TTWA

fig, ax = plt.subplots(2,3,figsize=(8,5))
with sns.plotting_context('talk', font_scale = 3):
    for t,col in enumerate(main_col16):
        i,j = np.unravel_index(t,(2,3))
        g = sns.boxplot(data = ttwa_info16.applymap(remove_london), y = col, x = 'Rank', ax = ax[i,j]) 
        ax[i,j].tick_params(axis = 'x', rotation = 30)
        ax[i,j].tick_params(axis = 'both', labelsize = 12)
        ax[i,j].set_xlabel(ax[i,j].get_xlabel(), fontsize = 13)
        ax[i,j].set_ylabel(ax[i,j].get_ylabel(), fontsize = 13)
    # now plot the regions
    i,j = np.unravel_index(t+1,(5,4))
plt.tight_layout()
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_by_ranking_and_stats16_pt1_select.png'))
    

In [ ]:
with sns.axes_style('darkgrid'):
    with sns.plotting_context('talk'):
        fig = plt.figure(figsize = (11,5))
        g = sns.catplot( x = 'Occupation code', y = 'Mean distance', 
                            data = distances_data,
                     linewidth = 2, kind = 'box')
        tmp = g.fig.set_size_inches((11,5))
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_by_2d-SOC.png'))

In [ ]:
# replot the boxplot for first digit occupations
with sns.axes_style('darkgrid'):
    with sns.plotting_context('talk'):
        fig = plt.figure(figsize = (11,5))
        g = sns.catplot( x = 'Occupation first digit', y = 'Mean distance', 
                            data = distances_data,
                     linewidth = 2, kind = 'box')
        g.fig.set_size_inches((11,5))
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_by_1d-SOC.png'))

In [ ]:
'''
Accent, Accent_r, Blues, Blues_r, BrBG, BrBG_r, BuGn, BuGn_r, BuPu, BuPu_r, CMRmap, CMRmap_r, Dark2, Dark2_r, GnBu,
GnBu_r, Greens, Greens_r, Greys, Greys_r, OrRd, OrRd_r, Oranges, Oranges_r, PRGn, PRGn_r, Paired, Paired_r, Pastel1,
Pastel1_r, Pastel2, Pastel2_r, PiYG, PiYG_r, PuBu, PuBuGn, PuBuGn_r, PuBu_r, PuOr, PuOr_r, PuRd, PuRd_r, Purples, 
Purples_r, RdBu, RdBu_r, RdGy, RdGy_r, RdPu, RdPu_r, RdYlBu, RdYlBu_r, RdYlGn, RdYlGn_r, Reds, Reds_r, Set1, Set1_r, 
Set2, Set2_r, Set3, Set3_r, Spectral, Spectral_r, Wistia, Wistia_r, YlGn, YlGnBu, YlGnBu_r, YlGn_r, YlOrBr, YlOrBr_r,
YlOrRd, YlOrRd_r, afmhot, afmhot_r, autumn, autumn_r, binary, binary_r, bone, bone_r, brg, brg_r, bwr, bwr_r,cividis,
cividis_r, cool, cool_r, coolwarm, coolwarm_r, copper, copper_r, cubehelix, cubehelix_r, flag, flag_r, gist_earth,
gist_earth_r, gist_gray, gist_gray_r, gist_heat, gist_heat_r, gist_ncar, gist_ncar_r, gist_rainbow, gist_rainbow_r, 
gist_stern, gist_stern_r, gist_yarg, gist_yarg_r, gnuplot, gnuplot2, gnuplot2_r, gnuplot_r, gray, gray_r, hot, hot_r,
hsv, hsv_r, icefire, icefire_r, inferno, inferno_r, jet, jet_r, magma, magma_r, mako, mako_r, nipy_spectral,
nipy_spectral_r, ocean, ocean_r, pink, pink_r, plasma, plasma_r, prism, prism_r, rainbow, rainbow_r, rocket, 
rocket_r, seismic, seismic_r, spring, spring_r, summer, summer_r, tab10, tab10_r, tab20, tab20_r, tab20b, tab20b_r,
tab20c, tab20c_r, terrain, terrain_r, twilight, twilight_r, twilight_shifted, twilight_shifted_r, viridis, viridis_r,
vlag, vlag_r, winter, winter_r
'''
print('all palettes')

In [ ]:
# for the next plots, I need the TTWA ordered based on their mean
IX_ttwa = np.argsort(np.mean(M_dist, axis = 0))
print(len(IX_ttwa))
ordered_ttwa = [select_ttwa[ii] for ii in IX_ttwa]
ordered_ttwa_names = [select_ttwa_names[ii] for ii in IX_ttwa]

In [ ]:
# try the last plot, that is plot all TTWA on the x axis and each occupation as a different line
plt.figure(figsize = (20,5))
with sns.plotting_context('paper'):
    g = sns.stripplot(x='TTWA name',y='Mean distance',hue='Occupation code',data = distances_data,
                 order = ordered_ttwa_names, size = 7, jitter = False, alpha = 0.5, palette = 'PuOr')
    g.set_xticklabels(g.get_xticklabels(),rotation=90)
    g.set_ylabel(g.get_ylabel(), fontsize = 15)
    g.set_xlabel(g.get_xlabel(), fontsize = 15)
    g.axes.tick_params(axis = 'y', labelsize =14)
    g.axes.tick_params(axis = 'x', labelsize =8)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize = 11)
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_ordered_2d-SOC.png'))

In [ ]:
# same plot as above, but group by first digit of SOC
plt.figure(figsize = (20,5))
with sns.plotting_context('paper'):
    g = sns.stripplot(x='TTWA name',y='Mean distance',hue='Occupation first digit',data = distances_data,
                 size = 7, jitter = False, order = ordered_ttwa_names, alpha = 0.5, 
                      palette = 'PuOr')
    g.set_xticklabels(g.get_xticklabels(),rotation=90)
    g.set_ylabel(g.get_ylabel(), fontsize = 15)
    g.set_xlabel(g.get_xlabel(), fontsize = 15)
    g.axes.tick_params(axis = 'y', labelsize =14)
    g.axes.tick_params(axis = 'x', labelsize =8)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize = 14, title = 'SOC code')
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_ordered_1d-SOC.png'))

In [ ]:
# same plot as above, but group by first digit of SOC and grey out points not belonging to the region of interest
def greyout_SOC_by_region(df,region):
    #for row in df.index:
    if df['Region'] != region:
        df['tmp'] = 10
    else:
        df['tmp'] = df['Occupation first digit']
    return df

# prepare the color palette:
palette = sns.color_palette(palette = 'PuOr',n_colors = 9)
palette = [(0.9,0.9,0.9)] + palette
df = copy.deepcopy(distances_data)
df['tmp'] = 1
with sns.plotting_context('paper'):
    for region in ['Wales','West Midlands', 'Scotland','London']:
        plt.figure(figsize = (20,5))
        g = sns.stripplot(x='TTWA name',y='Mean distance',hue='tmp',palette = palette,
                          data = df.apply(lambda x: greyout_SOC_by_region(x,region),axis = 1),
                     size = 7, jitter = False, order = ordered_ttwa_names, alpha = 0.5)
        g.set_xticklabels(g.get_xticklabels(),rotation=90)
        g.set_ylabel(g.get_ylabel(), fontsize = 15)
        g.set_xlabel(g.get_xlabel(), fontsize = 15)
        g.axes.tick_params(axis = 'y', labelsize =14)
        g.axes.tick_params(axis = 'x', labelsize =8)
        g.axes.set_title(region, fontsize = 15)
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize = 14, title = 'SOC code')
        if FIGSAVE:
            plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_ordered_1d-SOC_region{}.png'.format(
                region)))

In [ ]:
# same plot as above,but only plot the 2 fastest and slowest occupations (1st digit) per TTWA
# define occupations colors
occupations_colours = sns.color_palette('Set2',8)
# group again by TTWA
groups = distances_data.groupby('TTWA')

plt.figure(figsize = (20,5))
with sns.plotting_context('paper'):
    g = sns.pointplot(data= distances_data, x = 'TTWA name', y = 'Mean distance', 
                      ci = None, join = False, order = ordered_ttwa_names, color = 'k', scale = 0.5)
    g.set_xticklabels(g.get_xticklabels(),rotation=90)
    g.set_ylabel(g.get_ylabel(), fontsize = 15)
    g.set_xlabel(g.get_xlabel(), fontsize = 15)
    g.axes.tick_params(axis = 'y', labelsize =14)
    g.axes.tick_params(axis = 'x', labelsize =8)
    # get the fastest and slowest occupations by TTWA
    for ii,ttwa in enumerate(ordered_ttwa[::3]):
        group = groups.get_group(ttwa)
        # group again by first digit occupation, take the average mean distances, order them
        tmp =group.groupby('Occupation first digit').apply(
            np.mean).sort_values('Mean distance')
        #occ_order = np.argsort(group['Mean distance'])
        for pos in [0,1,6,7]:
            plt.plot(ii*3,tmp['Mean distance'].iloc[pos],'o', 
                 color = occupations_colours[int(tmp.index[pos])-1])
    # add the legend
    # create fake points for the occupations
    for pos in range(8):
        plt.plot(10,6, markersize = 0, 
                 color = occupations_colours[pos], label = '{}'.format(pos+1))
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize = 13, title = 'SOC code')
    g.set_xlim([-2,171])
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_ordered_1d-SOC_subset.png'))

In [ ]:
### for each occupation code (maybe 1st digit SOC), find those TTWAs for which they have a commuting distance that
# differ from the mean more than N standard deviations (presumably it will be N=2 or 3)
print('TODO!')
print(list(ttwa_info11.index))

In [ ]:
# compute various averages across regions
distances_grouped = distances_data.groupby('TTWA').agg(np.mean).reset_index()
distances_grouped['Region'] = distances_grouped.apply(lambda x: assign_region(x,ttwa_info16), axis = 1)
distances_grouped.head(n=3)

In [ ]:
# now, plot mean (or median) commuting distance for each TTWA on a map
fig,ax = plt.subplots(figsize = (5,8))
_ = draw_map(distances_grouped, 'Mean distance', 'viridis', 
         np.min(distances_data['Mean distance']), np.max(distances_data['Mean distance']), gb_filename,
         ni_filename, ttwa_shp_filename, roi_col = 'TTWA', shp_col = 'ttwa11cd', fig = fig, ax = ax,
         params={'SAVEFIG': False})
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_map.png'))
        
distances_grouped = distances_grouped.assign(Mean_distance_deviation = 
                         distances_grouped['Mean distance']/distances_grouped['Mean distance'].mean())

In [ ]:
import utils_pin
reload(utils_pin)
fig,ax = plt.subplots(figsize = (15,24))
_ = utils_pin.draw_map_and_landmarks(distances_grouped, 'Mean distance', 'viridis', 
         np.min(distances_data['Mean distance']), np.max(distances_data['Mean distance']), gb_filename,
         ni_filename, ttwa_shp_filename, roi_col = 'TTWA', shp_col = 'ttwa11cd', fig = fig, ax = ax,
         params={'SAVEFIG': False}, add_names = True)
if FIGSAVE or True:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_map_with_names.png'))


In [ ]:
'''
# replot the same, but divided by the overall mean
fig,ax = plt.subplots(figsize = (5,8))
_ = draw_map(distances_grouped, 'Mean_distance_deviation', 'viridis', 
     np.min(distances_grouped['Mean_distance_deviation']), np.max(distances_grouped['Mean_distance_deviation']), 
     gb_filename, ni_filename, ttwa_shp_filename, roi_col = 'TTWA', shp_col = 'ttwa11cd', fig = fig, ax = ax,
     params={'SAVEFIG': False, 'file_name': os.path.join(plot_save_dir,'avg_commuting_distance_deviation_map.png')})
'''
print('not needed')

### Location quotients

We can compute multiple location quotients - but in general we need to consider two axis of variations: 

- geographical location (either TTWA or region) and larger reference area (region or nation)
- commuting distance distribution across SOC codes

We can compute:
- which SOC code has the highest location quotient when comparing TTWAs in each region (whitin region differences)
- which SOC code has the highest LQ when comparing TTWAs in the whole of the UK
- which SOC code has the highes LQ when comparing regions against the national average


In [ ]:
def compute_lq(local_dist, reference_dist):
    '''
    local_dist is the matrix of average commuting distances per SOC codes per local area (TTWA x SOC codes)
    reference_dist is the vector of average commuting distsnces per SOC codes in the reference area (1 x SOC codes)
    It returns the LQ matrix (TTWA x SOC codes), as well as the SOC code with highest LQ per local area (TTWA x 0)
    '''
    local_ratio = local_dist/np.sum(local_dist, axis = 1, keepdims = True)
    reference_ratio = reference_dist/np.sum(reference_dist)
    local_lq = local_ratio/reference_ratio
    max_local_soc = np.argmax(local_lq, axis = 1)
    return local_lq, max_local_soc+1
    

In [ ]:
# compute the average commuting distance per SOC code per region
regional_average= distances_data.groupby(['Region','Occupation first digit']).agg(np.mean)['Mean distance']
# compute the national average per SOC code
national_average = distances_data.groupby(['Occupation first digit']).agg(np.mean)['Mean distance']

In [ ]:
# compute a location quotient for each first digit SOC code and each TTWA in a region when comparing to the
# regional average
all_lq_within = {}
for region,group in distances_data.groupby('Region'):
    # collect all TTWAs for each region, and take the average for each TTWA and Occupation first digit
    local_df = group[['TTWA','Mean distance','Occupation first digit']].groupby(
        ['TTWA','Occupation first digit']).agg(np.mean)
    ttwa_names, first_digits = local_df.index.levels
    local_matrix = np.reshape(np.array(local_df.values),(-1,8))
    LQ1,LQ2 = compute_lq(local_matrix,regional_average[region].values[:,np.newaxis].T)
    all_lq_within[region] = pd.DataFrame(np.concatenate((LQ1,LQ2[:,np.newaxis]),axis = 1), 
                       columns = list(first_digits.values)+['highest SOC'], index = ttwa_names)

for t, region in enumerate(all_lq_within):
    all_lq_within[region] = all_lq_within[region].reset_index()

# compute a location quotient for each first digit SOC code and each TTWA in a region when comparing to the
# national average
local_df = distances_data.groupby(['TTWA','Occupation first digit']).agg(np.mean)['Mean distance'].reset_index(
    ).pivot(index = 'TTWA',columns = 'Occupation first digit')
LQ1,LQ2 = compute_lq(np.array(local_df), national_average.values[:,np.newaxis].T)
all_lq_uk = pd.DataFrame(np.concatenate((LQ1,LQ2[:,np.newaxis]), axis =1), 
                         columns = list(first_digits.values)+['highest SOC'], index = local_df.index)

# compute a location quotient for each region, as compared with the national average
local_df = distances_data.groupby(['Region','Occupation first digit'])['Mean distance'].agg(np.mean
                            ).reset_index().pivot(index = 'Region',columns = 'Occupation first digit')
LQ1,LQ2 = compute_lq(np.array(local_df), national_average.values[:,np.newaxis].T)
all_lq_regions = pd.DataFrame(np.concatenate((LQ1,LQ2[:,np.newaxis]), axis =1), 
                         columns = list(first_digits.values)+['highest SOC'], index = local_df.index)

# join the latter with the former, by assigning the same SOC code to all TTWA in the same region
all_lq_uk = all_lq_uk.assign(highest_SOC_regional = -1)
for ttwa in all_lq_uk.index:
    all_lq_uk['highest_SOC_regional'].loc[ttwa] = all_lq_regions['highest SOC'].loc[
        ttwa_info16['Region/Country'].loc[ttwa]]
    

In [ ]:
# first, plot the easy ones
fig,ax = plt.subplots(figsize = (5,8))
_ = draw_map(all_lq_uk.reset_index(), 'highest SOC', 'viridis', 1, 9, gb_filename,
         ni_filename, ttwa_shp_filename, roi_col = 'TTWA', shp_col = 'ttwa11cd', fig = fig, ax = ax,
         params={'SAVEFIG': False})
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_map_lq_ttwa_vs_nation.png'))

In [ ]:
fig,ax = plt.subplots(figsize = (5,8))
_ = draw_map(all_lq_uk.reset_index(), 'highest_SOC_regional', 'viridis', 1, 9, gb_filename,
         ni_filename, ttwa_shp_filename, roi_col = 'TTWA', shp_col = 'ttwa11cd', fig = fig, ax = ax,
         params={'SAVEFIG': False})
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_map_lq_region_vs_nation.png'))

In [ ]:
fig, ax = plt.subplots(4,3, figsize = (12,20))
reload(utils_pin)
# prepare dictionary of shapefiles and subsets to pass (we only want to plot each region)
map_shp = {'Scotland': gb_filename, 'Wales': gb_filename}
regions_filename = map_files_folder + 'Regions_December_2018_EN_BFC/Regions_December_2018_EN_BFC.shp'
for region in ['North East','North West','Yorkshire and The Humber','East Midlands','West Midlands',
               'East of England','London','South East','South West']:
    map_shp[region] = regions_filename
print(map_shp[list(all_lq_within.keys())[0]])
for ix, region in enumerate(all_lq_within.keys()):
    i, j = np.unravel_index(ix,(4,3))
    if region=='NI':
        _ = utils_pin.draw_map(all_lq_within[region], 'highest SOC', 'viridis', 1, 10, gb_filename, 
                 ni_filename, ttwa_shp_filename, subset_outlines = [region],
                 roi_col = 'TTWA', shp_col = 'ttwa11cd', fig = fig, ax = ax[i,j],
                 params={'SAVEFIG': False})
    else:
        _ = utils_pin.draw_map(all_lq_within[region], 'highest SOC', 'viridis', 1, 10, gb_filename, #map_shp[region], 
                 None, ttwa_shp_filename, subset_outlines = [region],
                 roi_col = 'TTWA', shp_col = 'ttwa11cd', fig = fig, ax = ax[i,j],
                 params={'SAVEFIG': False})
    ax[i,j].set_title(region)

plt.tight_layout()
        
if FIGSAVE:
    plt.savefig(os.path.join(plot_save_dir,'avg_commuting_distance_map_lq_ttwa_vs_region.png'))

In [ ]:
'''
Check memory usage of all variables
'''
'''
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') 
        and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)
'''
print('Finished')